# Main programme avec interaction avec l'utilisateur



L'idée de ce dossier est de modéliser un CP avec interaction d'un décision-maker.

In [1]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [2]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# Interaction with the solver

data = []

with open('./example.data', 'r') as f:    
    for line in f:
        data.append(line.strip().split())

n = int(data[0][0])
m = int(data[0][1])

T_machine = []
T_duration = []

for i in range(1,n+1):
    for j in range(0, 2*m, 2):
        T_machine.append(int(data[i][j]))
        T_duration.append(int(data[i][j+1]))

duration = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        ind_machine = int(T_machine[i*m + j])
        duration[i][ind_machine] = T_duration[i*m + j]


# --------- Call Solver constructor in Solver.py

solver = Solver(data)

model = CpoModel() 

# --------- Create the model variables 

solver.create_variables(model, n, m, duration)


# ------------ Add objective function to the solver

# print("\nAdding objective function to the solver...")
# # print(variables[i][T_machine[i*m + m -1]])
# makespan = max([end_of(variables[i][T_machine[i*m + m -1]]) for i in range(n)])
# solver.add_constraint(model.minimize(makespan))
# print("Objective function added !")

# ------------ Solve the model

print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, 60, n, m, T_machine, duration)

# ------------ Display the result

for sol in msol:
    sol.write()
print("Model solved !")


# ---------------- Interaction with the user

print("\n--------Interaction with the user...---------")

print("\nCreating the user...")
user = User()
print("User created !")

print("\nClassing solutions...")	
user.classerSolutions(msol)
print("Solutions classed !")

print("\nCreating preferences...")
pref = user.getPreferences()
print("Preferences created !")

print("\nDisplaying preferences...")
# for sol in pref:
#     sol.write()

print("\nPreferences : ", pref)



--------Main program is loading...---------

Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Creating the model variables...
Model variables created !

Adding precedence constraints to the solver...


TypeError: 'CpoIntervalVar' object is not subscriptable

In [ ]:
###  -------------- Iteration of the solver with the preferences

# ----------------- Add the preferences to the model

tps_max = 100
tps = runtime
it_max = 5
it = 1

criterion = (tps < tps_max) and (it < it_max) 

while criterion :

    it += 1
    
    solver = Solver(data)
    model = CpoModel() 

    # stop = int(input("Do you want to continue ? True(1) or False(0)"))

    bb = integer_var(0,1)
    solver.add_variable(bb)
    bb=1

    # --------- Create the model variables 

    solver.create_variables(model, n, m, duration)

    list_variables = solver.get_variables()
    variables = list_variables[1]
    print(variables)

    for sol in pref:
        b = integer_var(0,1)
        solver.add_variable(b)
        b=0
        for i in range(n):
            for j in range(m):
                #a = model.interval_var(start = sol[variables[i][j]].start, end= sol[variables[i][j]].end, size=int(duration[i][j]), name="a{}{}".format(i,j))
                #b =max(b,logical_or((model.start_of(a) != model.start_of(variables[i][j])), (model.end_of(a) != model.end_of(variables[i][j]))))
                b =max(b,logical_or(sol[variables[i][j]].start != model.start_of(variables[i][j]), sol[variables[i][j]].end != model.end_of(variables[i][j])))
        b = (b!=0)
        bb = bb * b
    solver.add_constraint(bb==1)

# ------------ Solve the model

    print("\nSolving the model...")
    msol, nb_solution, runtime = solver.solve(model, 60, n, m, T_machine)

# ------------ Display the result

    for sol in msol:
        sol.write()
    print("Model solved !")


# ---------------- Interaction with the user

    print("\n--------Interaction with the user...---------")

    print("\nClassing solutions...")	
    user.classerSolutions(msol)
    print("Solutions classed !")

    print("\nCreating preferences...")
    pref = user.getPreferences()
    print("Preferences created !")

    print("\nDisplaying preferences...")
    # for sol in pref:
    #     sol.write()

    print("\nPreferences : ", pref)
    print(len(pref))

    tps += runtime

    criterion = (tps < tps_max) and (it < it_max)

    if (it == it_max):
        print("The user has reached the maximum number of iterations !")
    # elif not stop:
    #     print("The user has chosen his/her solution !")
    elif (tps >= tps_max):
        print("The user has reached the maximum time !")


print("\n--------End of the interaction with the user...---------")


Solver initialized !
intVar(0, 1)


TypeError: 'CpoIntVar' object is not subscriptable